In [4]:
import pymongo
from pymongo import MongoClient

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]

x = mycol.find_one()

print(x)

None


In [1]:

import cv2
import threading
#from kafka import KafkaProducer
import base64
import json
import numpy as np
import datetime
from bson import ObjectId
import os
import sys
import multiprocessing
# from datetime import datetime
import pickle
import redis
import time
import requests
from common_utils import *
# import config
raw_image_path = '/home/maini/main/cameraEngine/savedimages/'

import camera_module
import dummy_cam

def config_file():
    cmd = os.system('v4l2-ctl --list-device | grep /dev/video > indexes.txt')

    config = [

    ]

    fr = open('indexes.txt','r')

    data = fr.read()
    data = data.split('\n')

    # print(data)


    camera_id = []

    counter = 0
    c = 0
    ids_list = ['cam2','cam1','cam3','cam4']#,'9cam','10cam','2cam','4cam','5cam','6cam']
    # ids_list = ['1cam','3cam','7cam','8cam','6cam','5cam','4cam','9cam','2cam']
    # ids_list = ['1cam','3cam','7cam','8cam','8cam','5cam','4cam','9cam','3cam']


    for i in data:
        if i == '':
            continue
        if counter %2 == 0:

            i = i.replace('\t','')
            # print(i)
            camera_id.append(i)
            config.append({'camera_id':i,'camera_index':ids_list[c]})
            
            c += 1

        
        counter += 1

    return config



def singleton(cls):
    instances = {}
    def getinstance():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]
    return getinstance

@singleton
class MongoHelper:
    client = None
    def __init__(self):
        if not self.client:
            self.client = MongoClient(host='localhost', port=27017)

        self.db = self.client[settings.MONGO_DB]
        # if settings.DEBUG:
            # self.db.set_profiling_level(2)
        # placeholder for filter
    """
    def getDatabase(self):
        return self.db
    """

    def getCollection(self, cname, create=False, codec_options=None):
        _DB = "LIVIS"
        DB = self.client[_DB]
        return DB[cname]

@singleton
class CacheHelper():
    def __init__(self):
        self.redis_cache = redis.StrictRedis(host="localhost", port="6379", db=0, socket_timeout=1)
        print("REDIS CACHE UP!")

    def get_redis_pipeline(self):
        return self.redis_cache.pipeline()
    
    #should be {'key'  : 'value'} always
    def set_json(self, k, v):
        try:
            #k, v = list(dict_obj.items())[0]
            v = pickle.dumps(v)
            return self.redis_cache.set(k, v)
        except redis.ConnectionError:
            return None

    def get_json(self, key):
        try:
            temp = self.redis_cache.get(key)
            #print(temp)\
            if temp:
                temp= pickle.loads(temp)
            return temp
        except redis.ConnectionError:
            return None
        return None

    def execute_pipe_commands(self, commands):
        #TBD to increase efficiency can chain commands for getting cache in one go
        return None

def create_folder():
    main_path = raw_image_path
    date_folder = datetime.datetime.now().strftime('%y_%m_%d')
    date_path = main_path + date_folder
    part_folder = datetime.datetime.now().strftime('%H_%M_%S')
    part_path = date_path+'/'+part_folder
    print(date_path,os.path.exists(date_path))
    print(part_path,os.path.exists(part_path))
    try:
        if not(os.path.exists(date_path)):
            print('Creating Date Path')
            os.mkdir(date_path)
    except Exception as e:
        print(e)
    try:
        if not(os.path.exists(part_path)):
            print('Creating Part Path')
            os.mkdir(part_path)
        return part_path
    except Exception as e:
        print(e)

def camPreview(previewName, camID, preprocess=[]):
    print("previewName-------",previewName)
    # print(preprocess)
    frame_counter = 0

    
    # rakshith start
    # cam = cv2.VideoCapture((camID),cv2.CAP_V4L2)
    cam_data = camID.split(':')
    if cam_data[0] == 'baumer' :
        cam = camera_module.camera('baumer',cam_data[1])
    elif cam_data[0] == 'dummy2':
        cam = dummy_cam.cam2()
    elif cam_data[0] == 'dummy3':
        cam = dummy_cam.cam3()
    elif cam_data[0] == 'dummy4':
        cam = dummy_cam.cam4()

    # rakshith stop

    # cam = cv2.VideoCapture(int(camID))

    # pipeline = 'v4l2src device=/dev/video0 ! video/x-raw, framerate=30/1,width=1920,height=1080 ! videoconvert ! appsink'
    # width = 1920
    # height = 1080
    # width = 1280
    # height = 1080
    width = 1920
    height = 1080
    

    #rakshith start
    
    # if cam.isOpened():
    #     cam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    #     cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    #     start_frame_number = 20
    #     cam.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
    #     cam.set(cv2.CAP_PROP_FOURCC,cv2.VideoWriter_fourcc('M','J','P','G'))

    #     # cam.set(cv2.CAP_PROP_FPS, 9)
    #     # print(camID)
    #     # print(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    #     # print(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
    #     # cam.set(cv2.CAP_PROP_FOURCC,cv2.VideoWriter_fourcc('M','J','P','G'))
    #     # print(dir(cam))
    #     time.sleep(0.1)
    #     rval, frame = cam.read()
    #     print(frame.shape)
    #     # time.sleep(5)

    # else:
    #     print(previewName,'not responding')
    #     rval = False
    # print(previewName,rval)
    print(previewName)
    #rakshith stop

    

    total_count = 12
    # measured = False
    print('Programme Restarted------------------------------------------------------------------------------------------')

    while True:
        #rakshith start

        # rval, frame = cam.read()
        frame = cam.fetch_image()
        #rakshith stop
        # img = str(ObjectId()) + '.jpg'
        # b,g,r = cv2.split(frame)
        # frame = b
        
        # # frame = cv2.resize(frame,(1080,720))
        # cv2.imwrite(img, frame)
        
        # with open(img, 'rb') as f:
        #     im_b64 = base64.b64encode(f.read())
        # payload_video_frame = {"frame": str(im_b64)}
        # os.remove(img)

        # frame = cv2.resize(frame,(1920,1080))
        if previewName == 'cam3' and CacheHelper().get_json("measure_length"):
            # rval, frame = cam.read()
            CacheHelper().set_json('Stage2_Status',"started")
            part_path = create_folder()
            CacheHelper().set_json('part_path_cam_service',part_path)
            print('Part path',part_path)
            if not part_path:
                part_path = create_folder()
            # rval, frame = cam.read()
            # rval, frame = cam.read()
            cv2.imwrite('/home/maini/main/cameraEngine/savedimages/measurement.jpg', frame)
            CacheHelper().set_json('measurement', frame)
            # temp_start = datetime.datetime.now()
            # temp_count = 0
            # while 1:
                
            #     cv2.imwrite(part_path+'/measurement'+str(temp_count)+'.jpg', frame)
            #     temp_count += 1
            #     rval, frame = cam.read()
            #     if datetime.datetime.now() - temp_start >= datetime.timedelta(seconds=5):
            #         break
            cv2.imwrite(part_path+'/measurement.jpg', frame)
            cv2.imwrite('measurement.jpg', frame)
            print('\n\n\n\n\n\n\nMeasured\n\n\n\n\n\n')
            CacheHelper().set_json('measurement_complete',True)
            CacheHelper().set_json('measure_length',False)

        if CacheHelper().get_json(previewName+'_Start_Insp'):
            if frame_counter == 0:
                start = datetime.datetime.now()
                part_path = CacheHelper().get_json('part_path_cam_service')
                print('Part path',part_path)
                if not part_path:
                    part_path = create_folder()
                # if previewName=='cam3':
                #     cv2.imwrite('/home/maini/main/cameraEngine/savedimages/measurement.jpg', frame)
                #     CacheHelper().set_json('measurement', frame)
                #     cv2.imwrite(part_path+'/measurement.jpg', frame)
                #     print('\n\n\n\n\n\n\nMeasured\n\n\n\n\n\n')

            topic1=str(previewName)+"_"+str(frame_counter)
            #rakshith start

            # rval, frame = cam.read()
            frame = cam.fetch_image()
            #rakshith stop
            cv2.imwrite('/home/maini/main/cameraEngine/savedimages/raw_'+str(previewName)+'_'+str(frame_counter)+'.jpg', frame)

            cv2.imwrite(part_path+'/'+str(previewName)+'_'+str(frame_counter)+'.jpg', frame)

            CacheHelper().set_json(topic1, frame)
            print("topic1*********************",topic1)
            CacheHelper().set_json('Trigger_'+previewName,True)
            print(topic1,'Trig',frame_counter)
            frame_counter += 1
            
            if frame_counter==total_count:
                frame_counter = 0
                # measured = False
                # if previewName=='cam3':
                #     cv2.imwrite('/home/maini/main/cameraEngine/savedimages/measurement.jpg', frame)
                #     CacheHelper().set_json('measurement', frame)
                #     cv2.imwrite(part_path+'/measurement.jpg', frame)
                #     print('\n\n\n\n\n\n\nMeasured\n\n\n\n\n\n')
                print(previewName,"*****************complete****************")
                CacheHelper().set_json(previewName+'_completed',True)
                print(previewName,datetime.datetime.now() - start)
                # break
                CacheHelper().set_json(previewName+'_Start_Insp',False)
                break
            CacheHelper().set_json(previewName+'_Start_Insp',False)
        # break
    #rakshith start
    # cam.release()        
    #rakshith stop
    cv2.destroyAllWindows()

def reset():
    CacheHelper().set_json('cam1_completed',False)
    CacheHelper().set_json('cam2_completed',False)
    CacheHelper().set_json('cam3_completed',False)
    CacheHelper().set_json('cam4_completed',False)
    CacheHelper().set_json('measurement_complete',False)
    CacheHelper().set_json('measure_length',False)
    CacheHelper().set_json('cam1_Start_Insp',False)
    CacheHelper().set_json('cam2_Start_Insp',False)
    CacheHelper().set_json('cam3_Start_Insp',False)
    CacheHelper().set_json('cam4_Start_Insp',False)

def __main__():
# Create threads as follows
    print("main function")
    thread_pool = {}  
    # f = open('config.json', 'r')
    # distros_dict = json.load(f)
    distros_dict = config_file()
    print(distros_dict)
    # f.close()
    reset()
    print("after reset")
    for distro in distros_dict:  
        print("disto:::",distro)
        if 'preprocess' not in distro:
            distro['preprocess'] = [] 
        
        thread_pool[distro['camera_id']] = multiprocessing.Process(target=camPreview, args=(distro['camera_index'],distro['camera_id'], distro['preprocess']))
        # print()        
    for tt in thread_pool:
        # print('process',tt)
        thread_pool[tt].start()
        print(tt,"--------------------------------")
    for tt in thread_pool:
        thread_pool[tt].join()

    
    
if __name__ == "__main__":
    __main__() 
    cv2.destroyAllWindows()


d:\Anaconda\envs\mmnew\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda\envs\mmnew\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda\envs\mmnew\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'common_utils'

In [4]:

from livis.settings import MONGO_DB
import pymongo
def singleton(cls):
    instances = {}
    def getinstance():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]
    return getinstance

@singleton
class MongoHelper:
    client = None
    def __init__(self):
        if not self.client:
            self.client = pymongo.MongoClient(host='localhost', port=27017)

        self.db = self.client[MONGO_DB]
        # if settings.DEBUG:
            # self.db.set_profiling_level(2)
        # placeholder for filter
    """
    def getDatabase(self):
        return self.db
    """

    def getCollection(self, cname, create=False, codec_options=None):
        _DB = "LIVIS"
        DB = self.client[_DB]
        return DB[cname]


def list_specific_jig(jig_type):

    message = None
    status_code = None

    
   
    if jig_type is None:
        message = "jig_type not provided"
        status_code = 400
        return message,status_code

    try:
        mp = MongoHelper().getCollection('jig')
    except:
        message = "Cannot connect to db"
        status_code = 500
        return message,status_code
   
    p = [i for i in mp.find()]
   
    new_list = []
   
    for i in p:
       
        if i['jig_type'] == jig_type:
       
            new_list.append(i)
       
    return new_list,200

In [7]:
jig_type = 'AGILIS'
new_list = list_specific_jig(jig_type)
new_list

([{'_id': ObjectId('63be84976826d4edf77b1b54'),
   'jig_type': 'AGILIS',
   'oem_number': '0N-33405',
   'heatsink_match': 'HSC123',
   'palette_match': 'P-123',
   'kanban': [{'part_number': ['FGY100T65'],
     'position': 5,
     'part_type': 'IGBT'}],
   'vendor_match': ['1,15,4'],
   'is_deleted': False,
   'full_img': [{'cam_name': 'extreme_left_camera', 'regions': []},
    {'cam_name': 'left_camera', 'regions': []},
    {'cam_name': 'middle_camera', 'regions': []},
    {'cam_name': 'right_camera', 'regions': []},
    {'cam_name': 'extreme_right_camera', 'regions': []}]}],
 200)

In [7]:
def config_file1():
    config = []
    config.append({'camera_id':"baumer:192.168.1.4",'camera_index':"cam1"})
    config.append({'camera_id':"baumer:192.168.1.3",'camera_index':"cam2"})
    return config

In [2]:
user_details = {'user_id': 'b05cc995-b6b6-40ec-bd55-704c326339e0', 'username': 'anusha', 'role_name': 'admin'}
user_name = user_details['user_id']
print(type(user_name))

<class 'str'>


In [8]:
def config_file1():
    baumer_ip = []
    baumer_ip.append("192.168.1.4")
    baumer_ip.append("192.168.1.3")
    return baumer_ip

In [17]:
from common.utils import RedisKeyBuilderServer,CacheHelper,MongoHelper
import sys
import sys
sys.path.insert(0,"../livis/")
from settings import REDIS_CLIENT_HOST
from settings import REDIS_CLIENT_PORT
from settings import WORKSTATION_COLLECTION

original_frame_keyholder = 'original-frame'

mp = MongoHelper().getCollection(WORKSTATION_COLLECTION)

p = [p for p in mp.find()]

workstation_id = p[0]['_id']
print(workstation_id)

data = RedisKeyBuilderServer(workstation_id).workstation_info
cam_list = []
key_list = []
cap_list = []
for cam in data['camera_config']['cameras']:
    camera_index = cam['camera_id']
    camera_name = cam['camera_name']
    key = RedisKeyBuilderServer(workstation_id).get_key(cam['camera_id'],original_frame_keyholder) #WS-01_0_original-frame
    key_list.append(key)

5f4cce083f8fa13b3271656b


In [21]:
key_list[0]

'ws_01_1_original-frame'

In [19]:
key_list

['ws_01_1_original-frame', 'ws_01_2_original-frame']

In [34]:
from common import camera_module
baumer_ip = config_file1()
baumer_ip[0]
cam_1 = camera_module.camera('baumer',baumer_ip[0])

NotConnectedException: NotConnectedException: No device with '192.168.1.4' found.

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.